This script joins the master person table, srpatient table and IMD lookup table to get the data which is required for the palliative care master table. check cell [3] contains all the lates tables and then run script.

In [1]:
# Import libraries.
import pandas as pd
from google.cloud import bigquery
from google.cloud.bigquery import LoadJobConfig
from google.cloud.bigquery import SchemaField

In [2]:
client = bigquery.Client()

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
## CHECK: these will need to be replaced with the latest table.

master_person = "yhcr-prd-phm-bia-core.CB_FDM_MASTER.person"
SRPatient = "yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srpatient"
IMD_lookup = "yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_IMD_2019"


The rest of this script can be run without making any changes

In [4]:
#delete the existing table 1 before creating a new table1 
table_id = 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1'
client.delete_table(table_id, not_found_ok=True)  
print("Deleted table '{}'.".format(table_id))

Deleted table 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1'.


In [5]:
## update this table - yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_V8.tbl_srpatient with the latest srpatient table
##the other two tables used are yhcr-prd-phm-bia-core.CB_FDM_MASTER.person which doesn't need updating 
## and yhcr-prd-phm-bia-core.CB_LOOKUPS.tbl_IMD_2019 which is used to get IMD from the LSOA, this is the latest IMD table, if there is an update to this IMD table it will need updating in the script
palliative_care1 = f"""
select distinct person_id, datebirth, datedeath,
-- Year of death
Extract(Year from datedeath) as Year_of_death,
-- Age
date_diff(datedeath, datebirth, Year) as Age_at_death,
gender,
GP_Practice,
ethnicity,
LSOA,
IMD

from

(select distinct b1.*, b2.IMD

from

(select distinct a1.*, a2.* EXCEPT(person_id)

from

(select distinct person_id, extract(date from birth_datetime) AS datebirth, extract(date from death_datetime) AS datedeath, ethnicity_source_value as ethnicity
from {master_person}
)a1

left join

(SELECT distinct person_id, gender, idorganisationvisibleto as GP_Practice ,soa as LSOA 

FROM  {SRPatient}
)a2

on a1.person_id =a2.person_id


)b1 

left join 
-- Use SOA from primary care table to get IMD from lookup table
(select distinct lsoa_code_2011, cast(index_of_multiple_deprivation_imd_decile_where_1_is_most_depr as INT64) as IMD

from {IMD_lookup}
)b2

on b1.LSOA = b2.lsoa_code_2011
)

"""


In [6]:
palliative_Care_table1 = pd.read_gbq(palliative_care1)

/opt/conda/lib/python3.7/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
##Create table in Bigquery to store table 1 temporarily
table1 = 'yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1'
##Load the table into BigQuery
job1 = client.load_table_from_dataframe(palliative_Care_table1, table1)

 check this table is present  in the dataset - yhcr-prd-phm-bia-core.CB_FDM_Palliative_care_v1.tmp_table1